In [14]:
import pandas as pd
import tensorflow as tf

In [15]:
# Column names are the suit and rank of five cards in order and its class
COLUMN_NAMES = ['suit1', 'rank1', 'suit2', 'rank2', 'suit3', 'rank3', 'suit4', 'rank4', 'suit5', 'rank5', 'poker_hand']

# The class 'poker hand' contains ten situations
POKER_HANDS = [ 'Nothing', 'One pair', 'Two pairs', 'Three of a kind',  'Straight', 'Flush', 'Full house', 'Four of a kind', 'Straight flush', 'Royal flush']

In [16]:
# Load data and pre-processing data sets
def load_data(y_name='poker_hand'):
    train_path = './data/poker-hand-training-true.data'
    test_path = './data/poker-hand-testing.data'

    # Segment the labels and features of data sets by selecting the first ten columns as x and the last column as y
    train = pd.read_csv(train_path, names=COLUMN_NAMES, header=None)
    train_x, train_y = train, train.pop(y_name)
    test = pd.read_csv(test_path, names=COLUMN_NAMES, header=None)
    test_x, test_y = test, test.pop(y_name)

    return train_x, train_y, test_x, test_y

In [17]:
# Training dataset
def train_input_fn(features, labels, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)
    return dataset

In [18]:
# Evaluation
def eval_input_fn(features, labels, batch_size):
    features = dict(features)
    if labels is None:
        inputs = features
    else:
        inputs = (features, labels)

    dataset = tf.data.Dataset.from_tensor_slices(inputs)
    dataset = dataset.batch(batch_size)
    return dataset

In [19]:
# Exceptional cases for Flush, Straight flush and Royal flush
def exceptional_case(predict_dict):
    flush = []
    straight_flush = []
    royal_flush = []
    for i in range(len(predict_dict['suit1'])):
        predict_set = (predict_dict['rank1'][i], predict_dict['rank2'][i], predict_dict['rank3'][i], predict_dict['rank4'][i], predict_dict['rank5'][i],)
        predict_list = list(predict_set)
        
        # Flush: five cards with the same suit
        if predict_dict['suit1'][i] == predict_dict['suit2'][i] == predict_dict['suit3'][i] == predict_dict['suit4'][i] == predict_dict['suit5'][i]:
            #  If rank of five cards are not repeated, sort them
            if len(predict_list) == 5:
                predict_list.sort()
                # Royal flush: {Ace, King, Queen, Jack, Ten} + flush  
                if predict_list[0] == 1 and predict_list[1] == 10:
                    royal_flush.append(i)
                # Straight flush: five cards are sequentially ranked with no gaps + flush
                elif predict_list[4]-predict_list[0] == 4:
                    straight_flush.append(i)
                else:
                    flush.append(i)
            else:
                flush.append(i)

    return flush, straight_flush, royal_flush

In [26]:
# Load data
train_x, train_y, test_x, test_y = load_data()

feature_columns = []
for key in train_x.keys():
    feature_columns.append(tf.feature_column.numeric_column(key=key))

# DNN classifier
classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns, hidden_units=[100,100,100,100,100], n_classes=10)

# Training
classifier.train( input_fn=lambda:train_input_fn(train_x, train_y, 100), steps=50000)

# Evaluation
eval_result = classifier.evaluate(input_fn=lambda:eval_input_fn(test_x, test_y, 100))

# Test
suit1 = []
rank1 = []
suit2 = []
rank2 = []
suit3 = []
rank3 = []
suit4 = []
rank4 = []
suit5 = []
rank5 = []
expected = []

for x in range(len(test_x)):
    suit1.append(test_x.values[x][0])
    rank1.append(test_x.values[x][1])
    suit2.append(test_x.values[x][2])
    rank2.append(test_x.values[x][3])
    suit3.append(test_x.values[x][4])
    rank3.append(test_x.values[x][5])
    suit4.append(test_x.values[x][6])
    rank4.append(test_x.values[x][7])
    suit5.append(test_x.values[x][8])
    rank5.append(test_x.values[x][9])
    expected.append(test_y.values[x])

predict_x = {'suit1': suit1, 'rank1': rank1, 'suit2': suit2, 'rank2': rank2, 'suit3': suit3, 'rank3': rank3, 'suit4': suit4, 'rank4': rank4, 'suit5': suit5, 'rank5': rank5,}

# Prediction
predictions = classifier.predict( input_fn=lambda: eval_input_fn(predict_x, labels=None, batch_size=100))

# To deal with Flush, Straight flush, Royal flush separately
flush, straight_flush, royal_flush = exceptional_case(predict_x)

# Count the total number of accurate predictions
count_all = 0
# Count the number of each class
count_kind = [0,0,0,0,0,0,0,0,0,0]
# Count the accurate number of each class
count_right = [0,0,0,0,0,0,0,0,0,0]
for i, pred_dict, expec in zip(range(len(expected)), predictions, expected):
    if i in flush:
        class_id = 5
        probability = 1
    elif i in straight_flush:
        class_id = 8
        probability = 1
    elif i in royal_flush:
        class_id = 9
        probability = 1
    else:
        class_id = pred_dict['class_ids'][0]
        probability = pred_dict['probabilities'][class_id]

    #template = ('\nPrediction is "{}" ({:.3f}%), expected "{}"')
    #print(template.format(POKER_HANDS[class_id], 100 * probability, POKER_HANDS[expec]))

    # Compute accuracy
    count_kind[expec] += 1
    if class_id == expec:
        count_all += 1
        count_right[class_id] += 1
count_prop = [float(a)/float(b) for a,b in zip(count_right,count_kind)]


print('\n-------------------------------------------------------------------')
print('\nAccuracy of trained model： {accuracy:0.5f}'.format(**eval_result))

print('\n-------------------------------------------------------------------')
print('\nNumber of test data： ',len(expected))
print('\nNumber of accurate predictions： ',count_all)
print('\nAccuracy of test dataset： ',float(count_all)/len(expected))

print('\n-------------------------------------------------------------------')
print('\nNumber of each class： ',count_kind)
print('\nNumber of accurate predictions in each class： ',count_right)
print('\nAccuracy of each class： ',count_prop)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/8z/n6_p2vn94r32nmg9m7ldlp6h0000gn/T/tmp5_g05iih', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0xb323f9550>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create Chec

INFO:tensorflow:loss = 72.75125, step = 7001 (0.212 sec)
INFO:tensorflow:global_step/sec: 464.455
INFO:tensorflow:loss = 59.477283, step = 7101 (0.215 sec)
INFO:tensorflow:global_step/sec: 453.223
INFO:tensorflow:loss = 62.75462, step = 7201 (0.221 sec)
INFO:tensorflow:global_step/sec: 434.344
INFO:tensorflow:loss = 75.23703, step = 7301 (0.230 sec)
INFO:tensorflow:global_step/sec: 442.539
INFO:tensorflow:loss = 69.863304, step = 7401 (0.226 sec)
INFO:tensorflow:global_step/sec: 385.923
INFO:tensorflow:loss = 65.58957, step = 7501 (0.260 sec)
INFO:tensorflow:global_step/sec: 368.593
INFO:tensorflow:loss = 55.84808, step = 7601 (0.271 sec)
INFO:tensorflow:global_step/sec: 412.75
INFO:tensorflow:loss = 70.853355, step = 7701 (0.242 sec)
INFO:tensorflow:global_step/sec: 428.209
INFO:tensorflow:loss = 72.57912, step = 7801 (0.233 sec)
INFO:tensorflow:global_step/sec: 453.315
INFO:tensorflow:loss = 60.802925, step = 7901 (0.220 sec)
INFO:tensorflow:global_step/sec: 414.303
INFO:tensorflow:l

INFO:tensorflow:loss = 23.794401, step = 15301 (0.212 sec)
INFO:tensorflow:global_step/sec: 468.982
INFO:tensorflow:loss = 35.133347, step = 15401 (0.213 sec)
INFO:tensorflow:global_step/sec: 479.996
INFO:tensorflow:loss = 31.176765, step = 15501 (0.208 sec)
INFO:tensorflow:global_step/sec: 481.666
INFO:tensorflow:loss = 32.19273, step = 15601 (0.208 sec)
INFO:tensorflow:global_step/sec: 475.855
INFO:tensorflow:loss = 22.757078, step = 15701 (0.210 sec)
INFO:tensorflow:global_step/sec: 480.125
INFO:tensorflow:loss = 26.54527, step = 15801 (0.208 sec)
INFO:tensorflow:global_step/sec: 476.406
INFO:tensorflow:loss = 20.027155, step = 15901 (0.210 sec)
INFO:tensorflow:global_step/sec: 474.816
INFO:tensorflow:loss = 38.709488, step = 16001 (0.211 sec)
INFO:tensorflow:global_step/sec: 452.929
INFO:tensorflow:loss = 26.433903, step = 16101 (0.221 sec)
INFO:tensorflow:global_step/sec: 455.038
INFO:tensorflow:loss = 15.921997, step = 16201 (0.220 sec)
INFO:tensorflow:global_step/sec: 456.084
IN

INFO:tensorflow:global_step/sec: 223.097
INFO:tensorflow:loss = 10.174167, step = 23601 (0.452 sec)
INFO:tensorflow:global_step/sec: 359.921
INFO:tensorflow:loss = 10.999934, step = 23701 (0.274 sec)
INFO:tensorflow:global_step/sec: 388.644
INFO:tensorflow:loss = 9.583777, step = 23801 (0.257 sec)
INFO:tensorflow:global_step/sec: 355.817
INFO:tensorflow:loss = 8.175148, step = 23901 (0.281 sec)
INFO:tensorflow:global_step/sec: 294.56
INFO:tensorflow:loss = 13.043217, step = 24001 (0.339 sec)
INFO:tensorflow:global_step/sec: 408.005
INFO:tensorflow:loss = 5.1115484, step = 24101 (0.245 sec)
INFO:tensorflow:global_step/sec: 250.452
INFO:tensorflow:loss = 12.658482, step = 24201 (0.399 sec)
INFO:tensorflow:global_step/sec: 367.702
INFO:tensorflow:loss = 6.5507107, step = 24301 (0.273 sec)
INFO:tensorflow:global_step/sec: 381.711
INFO:tensorflow:loss = 7.4994006, step = 24401 (0.261 sec)
INFO:tensorflow:global_step/sec: 464.535
INFO:tensorflow:loss = 10.676665, step = 24501 (0.215 sec)
INF

INFO:tensorflow:global_step/sec: 457.07
INFO:tensorflow:loss = 2.3052177, step = 31901 (0.219 sec)
INFO:tensorflow:global_step/sec: 472.404
INFO:tensorflow:loss = 4.109937, step = 32001 (0.212 sec)
INFO:tensorflow:global_step/sec: 476.444
INFO:tensorflow:loss = 8.208822, step = 32101 (0.210 sec)
INFO:tensorflow:global_step/sec: 471.627
INFO:tensorflow:loss = 4.7328634, step = 32201 (0.212 sec)
INFO:tensorflow:global_step/sec: 470.119
INFO:tensorflow:loss = 1.2432246, step = 32301 (0.213 sec)
INFO:tensorflow:global_step/sec: 470.066
INFO:tensorflow:loss = 1.5939206, step = 32401 (0.213 sec)
INFO:tensorflow:global_step/sec: 473.16
INFO:tensorflow:loss = 5.9579124, step = 32501 (0.211 sec)
INFO:tensorflow:global_step/sec: 466.947
INFO:tensorflow:loss = 2.3845496, step = 32601 (0.214 sec)
INFO:tensorflow:global_step/sec: 476.177
INFO:tensorflow:loss = 4.028207, step = 32701 (0.210 sec)
INFO:tensorflow:global_step/sec: 475.181
INFO:tensorflow:loss = 5.36403, step = 32801 (0.210 sec)
INFO:te

INFO:tensorflow:loss = 1.4524679, step = 40101 (0.243 sec)
INFO:tensorflow:global_step/sec: 454.071
INFO:tensorflow:loss = 1.8405874, step = 40201 (0.221 sec)
INFO:tensorflow:global_step/sec: 440.296
INFO:tensorflow:loss = 4.897618, step = 40301 (0.227 sec)
INFO:tensorflow:global_step/sec: 444.676
INFO:tensorflow:loss = 1.2573098, step = 40401 (0.225 sec)
INFO:tensorflow:global_step/sec: 449.75
INFO:tensorflow:loss = 1.2301159, step = 40501 (0.222 sec)
INFO:tensorflow:global_step/sec: 468.496
INFO:tensorflow:loss = 0.7984688, step = 40601 (0.214 sec)
INFO:tensorflow:global_step/sec: 374.568
INFO:tensorflow:loss = 0.8266161, step = 40701 (0.268 sec)
INFO:tensorflow:global_step/sec: 460.857
INFO:tensorflow:loss = 2.5202599, step = 40801 (0.216 sec)
INFO:tensorflow:global_step/sec: 422.169
INFO:tensorflow:loss = 1.122458, step = 40901 (0.237 sec)
INFO:tensorflow:global_step/sec: 447.78
INFO:tensorflow:loss = 0.75569385, step = 41001 (0.223 sec)
INFO:tensorflow:global_step/sec: 420.129
INF

INFO:tensorflow:loss = 3.2699678, step = 48301 (0.214 sec)
INFO:tensorflow:global_step/sec: 471.801
INFO:tensorflow:loss = 2.3487074, step = 48401 (0.212 sec)
INFO:tensorflow:global_step/sec: 475.35
INFO:tensorflow:loss = 0.32394913, step = 48501 (0.210 sec)
INFO:tensorflow:global_step/sec: 457.118
INFO:tensorflow:loss = 0.48598033, step = 48601 (0.219 sec)
INFO:tensorflow:global_step/sec: 465.297
INFO:tensorflow:loss = 0.6273528, step = 48701 (0.215 sec)
INFO:tensorflow:global_step/sec: 472.46
INFO:tensorflow:loss = 0.45093167, step = 48801 (0.211 sec)
INFO:tensorflow:global_step/sec: 474.469
INFO:tensorflow:loss = 0.6386316, step = 48901 (0.211 sec)
INFO:tensorflow:global_step/sec: 469.821
INFO:tensorflow:loss = 0.35786214, step = 49001 (0.213 sec)
INFO:tensorflow:global_step/sec: 475.794
INFO:tensorflow:loss = 0.6464147, step = 49101 (0.210 sec)
INFO:tensorflow:global_step/sec: 477.37
INFO:tensorflow:loss = 0.3783582, step = 49201 (0.210 sec)
INFO:tensorflow:global_step/sec: 396.206